<a href="https://colab.research.google.com/github/mmcinnestaylor/NMA-CN-2022/blob/main/project/steinmetz_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
import numpy as np

from datetime import datetime
from tqdm import tqdm

In [3]:
BASE_PATH = '/content/drive/MyDrive/steinmetz' # Path to dataset base directory

WINDOW_SIZE = 10 #will be used for binning spikes

In [17]:
# @title Helper Functions
#@markdown - bin_spikes()
def bin_spikes(spike_times, window_size):
  pass

## Load Data

In [ ]:
# @title Helper Functions
#@markdown - Uses the path defined in BASE_PATH
#@markdown - Assumes each recording session resides in a separate subdirectory formatted as mouseID_year-month-date

all_session_data = dict()

# Order sessions by surname + date
for i,session in tqdm(enumerate(sorted(os.listdir(BASE_PATH)))):
  session_path = os.path.join(BASE_PATH, session)

  print(f"Loading session: {session} ")

  # Sort spikes by cluster
  raw_spikes = np.load(session_path+'/spikes.times.npy')
  raw_clusters = np.load(session_path+'/spikes.clusters.npy')
  num_clusters = raw_clusters.max() 

  sorted_spike_times = [[] for i in range(num_clusters+1)]

  # Group spike times by their predicted cluster number 
  for j in range(len(raw_spikes)):
    sorted_spike_times[raw_clusters[j][0]].append(raw_spikes[j][0])
  
  # Convert clustered spikes to numpy arrays for efficiency
  for j,cluster in enumerate(sorted_spike_times):
    sorted_spike_times[j] = np.array(sorted_spike_times[j])

  # Load session data into dictionary
  all_session_data[i] = {
    # Session information
    'mouse_id': session.split('_')[0],
    'session_date': datetime.strptime(session.split('_')[1], '%Y-%m-%d').date(),

    # Neural Data 
    'clustered_spikes': sorted_spike_times,

    # Cluster Data
    'spike_clusters_annotations': np.load(session_path+'/clusters._phy_annotation.npy'), #cluster quality
    'clusters_peak_channels': np.load(session_path+'/clusters.peakChannel.npy'),

    # Channels Data
    #'channels_site': np.load(session_path+'/channels.site.npy')

  }
  



In [ ]:
# @title File writer for Kobayashi method

#num_clusters = len(sorted_spike_times.keys())

with open("/content/drive/MyDrive/steinmetz/Cori_2016-12-14/kobayashi_datafile_5n.txt", "w") as f:
  for i,cluster in enumerate(sorted_spike_times.keys()):
    written = False
    
    # check if cluster is valid 
    if spike_clusters_annotations[cluster][0] < 2:
      continue

    for time in sorted_spike_times[cluster]:
      # limit spikes to first 5 minutes
      if time <= 10:
        written = True
        f.write(str(time)+'\n')

    #if i != num_clusters:
    # append only if a spike time has been written for a given neuron
    if written:    
      f.write(';\n')
  
  f.close()